In [16]:
# importing required libraries

import os
import requests

import tqdm
import httpimport
import pinecone
import numpy as np
from PIL import Image

import torch
from torchvision import transforms

from dotenv import load_dotenv
import json


In [17]:
# defining constants
LECTURE_SLIDES_DIR = os.path.join(os.getcwd(), "lecture_slides")

# load API keys from globally-availabe .env file
load_dotenv(dotenv_path='/mnt/project/chatbotai/huggingface_cache/internal_api_keys.env', override=True)


True

In [40]:
# loading image dataset

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img_folder_path = "/mnt/project/chatbotai/asmita/main_fn/lecture_slides"
folder_list = os.listdir(img_folder_path)
print("folder count = ", len(folder_list))
count_images = 0
image_dict = {}

for folder in folder_list:
    file_list = os.listdir(img_folder_path+ "/" + folder)
    image_dict[folder] = len(file_list)
    count_images += len(file_list)

print(count_images)
print(sum(image_dict.values()))

folder count =  86
1749
1749


In [20]:
# initiate pinecone
pinecone_index = pinecone.Index("uiuc-chatbot-images")
pinecone.init(api_key=os.environ['PINECONE_API_KEY_FOR_IMAGES'], environment=os.environ['PINECONE_ENVIRONMENT_FOR_IMAGES'])
print(pinecone.list_indexes())

['uiuc-chatbot-images']


In [19]:
#pinecone.create_index("uiuc-chatbot-images", dimension=1000)

In [21]:
# Load pre-trained ResNet CNN
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
model.eval()

# Define image transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

Using cache found in /home/avd6/.cache/torch/hub/pytorch_vision_v0.6.0
/home/avd6/utils/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/avd6/utils/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:

img_folder_path = "/mnt/project/chatbotai/asmita/main_fn/lecture_slides"
folder_list = os.listdir(img_folder_path)
images_to_upsert = []
img_embedding = []

for folder in folder_list:
    image_list = os.listdir(img_folder_path+ "/" + folder)
    
    for i in image_list:
        path = img_folder_path + "/" + folder + "/" + i

        # create image id
        image_name = i.split(".")[0]
        img_id = folder + "_" + image_name

        # create image metadata
        img_metadata = {'lec': folder, 'image': image_name}

        # load image and preprocess
        img = Image.open(path)
        img = transform(img)
        img = img.unsqueeze(0)

        # generate image embedding
        with torch.no_grad():
            features = model(img)
        embedding = features.flatten().numpy().tolist()
        
        img_tuple = (img_id, embedding)
        #print(img_tuple)
        pinecone_index.upsert(vectors=[img_tuple])


In [23]:
pinecone.describe_index("uiuc-chatbot-images")

IndexDescription(name='uiuc-chatbot-images', metric='cosine', replicas=1, dimension=1000.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [33]:
# query the database

test_img_path = "/mnt/project/chatbotai/asmita/main_fn/lecture_slides/002/Slide4.jpeg"

# load image and preprocess
test_img = Image.open(test_img_path)
test_img = transform(test_img)
test_img = test_img.unsqueeze(0)

# generate image embedding
with torch.no_grad():
    features = model(img)
embedding = features.flatten().numpy().tolist()

retrieved_embeddings = pinecone_index.query(vector=embedding, top_k=3, include_values=True)

In [35]:
retrieved_embeddings['matches'][0]

{'id': '050_Slide3',
 'score': 1.0,
 'values': [-0.781775594,
            -0.552785218,
            -0.884783506,
            -0.069625929,
            1.27279091,
            1.85853446,
            -0.79017365,
            -1.04283,
            -1.58041811,
            -1.07749581,
            0.158826679,
            -1.30376554,
            -1.57733393,
            -0.505276918,
            0.287439,
            -0.369661063,
            0.561906636,
            -1.44291568,
            0.297690123,
            -0.750911057,
            0.3619726,
            -0.0221248437,
            0.341160297,
            0.326977462,
            -0.921638,
            -0.654015481,
            1.11509717,
            1.41422606,
            -0.225585923,
            0.819587529,
            1.1490016,
            -1.06266189,
            0.472304881,
            0.506290674,
            0.412800938,
            1.06824696,
            0.473063439,
            -0.0641223863,
            0.8675